In [1]:
from ohlc import AmberData
import asyncio
import aiohttp
import logging
import json
import datetime as dt
import pandas as pd
import requests
import time
from datetime import datetime
import numpy as np
from warnings import simplefilter
from create_rvFeatures import generate_rv_features

In [2]:
data_start = datetime.now()

now = datetime.now()
start = dt.datetime(2018, 1, 1)
end = dt.datetime.combine(dt.date.today(), dt.time.min)


ticker = "BTCUSDT"
interval_in = 'hours'
exchange_in="binance", 
changes =[1,3,6,12,24]
ewm_days = [3,7,21]
vol_windows=[1,3,7,15,30,60]
future=True
day_step = 1
if interval_in == "hours":
    day_step = 24
elif interval_in == "minutes":
    day_step = 24*60
hour_step = day_step/24
df = generate_rv_features(start, end, ticker, interval_in,exchange_in="binance", changes =[1,3,6,12,24], ewm_days = [3,7,21],vol_windows=[1,3,7,15,30,60], future=True)
data_end = datetime.now()
print("Gather RV", data_end-data_start)

Gather RV 0:00:33.148339


In [3]:
#%%get up down or chop based on sdev
df['closeChg%_forward24HR'] = df['closeChg%24HR'].shift(-int(day_step))
df['closeChg%_forward1HR'] = df['closeChg%1HR'].shift(-int(hour_step))
df_test = df[df['dt']> datetime(2022,12,1)]
df_train = df[df['dt'] <= datetime(2022,12,1)] 
std_24 = df_train['closeChg%_forward24HR'].std()
std_1 = df_train['closeChg%_forward1HR'].std()
def helperLabel(x, std_val):
  if abs(x) >= std_val:
      if x > 0:
          return 1
      else:
          return -1
  else:
      return 0
df['UpDownPred24HR'] = df['closeChg%_forward24HR'].apply(lambda x: helperLabel(x, std_24))
df['UpDownPred1HR'] = df['closeChg%_forward1HR'].apply(lambda x: helperLabel(x, std_1))
df_test = df[df['dt']> datetime(2022,12,1)]
df_train = df[df['dt'] <= datetime(2022,12,1)] 
df_test.reset_index(drop = True, inplace=True)  
df_train.reset_index(drop = True, inplace=True)  

In [4]:
start_class = datetime.now()
from pycaret.classification import *


exp_clf24 = setup(df_train,target='UpDownPred24HR',
        ignore_features=['dt', 'exchange', 'timestamp', 'instrument', 'closeChg%_forward1HR', 'closeChg%_forward24HR', 'UpDownPred1HR'],session_id=11,
        profile=False,  use_gpu=True,  normalize = True,  remove_multicollinearity=True) 
end_setup_class= datetime.now()
models_class = compare_models(turbo=True, n_select =4)
model_df_class = pull()
best_models_class = model_df_class.iloc[0:4]
end_class = datetime.now()

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recomp

,Description,Value
0,Session id,11
1,Target,UpDownPred24HR
2,Target type,Multiclass
3,Target mapping,"-1: 0, 0: 1, 1: 2"
4,Original data shape,"(26822, 1380)"
5,Transformed data shape,"(26822, 130)"
6,Transformed train set shape,"(18775, 130)"
7,Transformed test set shape,"(8047, 130)"
8,Ignore features,7
9,Numeric features,1372


[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8969,0.9585,0.8969,0.8988,0.8886,0.6844,0.7049,0.9330
lightgbm,Light Gradient Boosting Machine,0.8929,0.9523,0.8929,0.8946,0.8839,0.6705,0.6924,0.7370
rf,Random Forest Classifier,0.8903,0.9544,0.8903,0.8943,0.8796,0.6559,0.6837,0.9170
catboost,CatBoost Classifier,0.8863,0.9444,0.8863,0.8893,0.8752,0.6433,0.6709,0.6640
dt,Decision Tree Classifier,0.8698,0.8275,0.8698,0.8707,0.8701,0.6577,0.6579,0.6800
knn,K Neighbors Classifier,0.8089,0.8132,0.8089,0.7883,0.7870,0.3907,0.4120,1.4600
gbc,Gradient Boosting Classifier,0.8079,0.8438,0.8079,0.8225,0.7544,0.2580,0.3617,1.3300
ridge,Ridge Classifier,0.7733,0.0000,0.7733,0.7110,0.6847,0.0423,0.1023,0.6820
lr,Logistic Regression,0.7722,0.6718,0.7722,0.6978,0.6917,0.0689,0.1243,1.0110
ada,Ada Boost Classifier,0.7720,0.6878,0.7720,0.7156,0.6895,0.0560,0.1105,0.7460


In [5]:
#%%
print("compare_models", end_class-start_class)
evaluate_model(models_class[0])

compare_models 0:02:37.490002


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [6]:
#%%SAVE MODEL
best_class= create_model(best_models_class.index[0])
save_model(best_class,"24hr_up_down")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8962,0.9435,0.8962,0.8995,0.8867,0.6783,0.7029
1,0.8940,0.9606,0.8940,0.8961,0.8854,0.6746,0.6961
2,0.8962,0.9587,0.8962,0.8971,0.8881,0.6841,0.7030
3,0.9004,0.9615,0.9004,0.9013,0.8930,0.6999,0.7165
4,0.8972,0.9562,0.8972,0.9016,0.8880,0.6810,0.7066
5,0.9062,0.9631,0.9062,0.9085,0.8995,0.7160,0.7339
6,0.8887,0.9664,0.8887,0.8905,0.8788,0.6544,0.6783
7,0.8972,0.9600,0.8972,0.8982,0.8892,0.6866,0.7056
8,0.8961,0.9528,0.8961,0.8982,0.8877,0.6810,0.7022


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('clean_column_names',
                  TransformerWrapper(exclude=None, include=None,
                                     transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+'))),
                 ('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     includ...
                  ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                       class_weight=None, criterion='gini',
                                       max_depth=None, max_features='sqrt',
                                       max_leaf_nodes=None, max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_samp

In [7]:
#%% get class labels REGRESSION
from pycaret.classification import *
classifier = load_model("24hr_up_down");
class_pred = predict_model(classifier, data=df_train)
print(class_pred.columns)
#get state labels

df_train['state_class'] = class_pred['prediction_label']

Transformation Pipeline and Model Successfully Loaded


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.9732,0.9971,0.9833,0.9723,0.9774,0.9268,0.9281


Index(['exchange', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'instrument', 'dt', 'volume_24HR',
       ...
       'vol_rogers_60D_ewm21DChg12HR', 'vol_rogers_60D_ewm21DChg%12HR',
       'vol_rogers_60D_ewm21DChg24HR', 'vol_rogers_60D_ewm21DChg%24HR',
       'closeChg%_forward24HR', 'closeChg%_forward1HR', 'UpDownPred1HR',
       'UpDownPred24HR', 'prediction_label', 'prediction_score'],
      dtype='object', length=1382)


In [8]:
from pycaret.regression import *
a=setup(df_train,target='closeChg%_forward24HR',
        ignore_features=['dt', 'exchange', 'timestamp', 'instrument', 'closeChg%_forward1HR',  'closeChg%_forward1HR',  'UpDownPred1HR', 'UpDownPred24HR' ]
        ,session_id=12,profile=False,  use_gpu=True,  normalize = True,  remove_multicollinearity=True, bin_numeric_features=['state_class']);
models = compare_models(exclude = ['tr', 'lightgbm'],turbo=True)
#pull error df
model_df = pull()
#get top 4 models to train
best_models = model_df.iloc[0:4]

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recomp

,Description,Value
0,Session id,12
1,Target,closeChg%_forward24HR
2,Target type,Regression
3,Original data shape,"(26822, 1381)"
4,Transformed data shape,"(26822, 132)"
5,Transformed train set shape,"(18775, 132)"
6,Transformed test set shape,"(8047, 132)"
7,Ignore features,8
8,Numeric features,1373
9,Preprocess,True


[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0086,0.0002,0.0129,0.8850,0.0119,1.0473,1.6060
rf,Random Forest Regressor,0.0095,0.0002,0.0139,0.8669,0.0128,1.0907,3.6350
catboost,CatBoost Regressor,0.0114,0.0002,0.0154,0.8351,0.0141,1.4861,3.2890
dt,Decision Tree Regressor,0.0123,0.0004,0.0192,0.7439,0.0148,2.4716,1.5570
gbr,Gradient Boosting Regressor,0.0155,0.0004,0.0209,0.6992,0.0191,1.2573,11.7440
ada,AdaBoost Regressor,0.0187,0.0006,0.0239,0.6049,0.0214,1.3756,4.7590
lr,Linear Regression,0.0175,0.0006,0.0240,0.6017,0.0207,1.5513,1.2480
ridge,Ridge Regression,0.0176,0.0006,0.0242,0.5962,0.0207,1.5859,0.9910
omp,Orthogonal Matching Pursuit,0.0176,0.0006,0.0242,0.5949,0.0210,1.4500,1.1080
br,Bayesian Ridge,0.0177,0.0006,0.0242,0.5941,0.0209,1.5568,1.9530


In [9]:
et = create_model('et')
evaluate_model(et)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0086,0.0002,0.0128,0.8967,0.0119,0.7698
1,0.0085,0.0002,0.0125,0.8977,0.0116,1.1406
2,0.0085,0.0002,0.0128,0.8875,0.0117,1.0828
3,0.0084,0.0002,0.0125,0.8920,0.0116,1.2342
4,0.0088,0.0002,0.0131,0.8883,0.0122,1.4101
5,0.0088,0.0002,0.0132,0.8769,0.0122,1.0979
6,0.0087,0.0002,0.0138,0.8734,0.0127,0.8853
7,0.0086,0.0002,0.0132,0.8662,0.0122,1.0604
8,0.0086,0.0002,0.0126,0.8868,0.0115,0.9542


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…